In [7]:
!pip install neuralforecast
# !pip install torchinfo
!pip install codecarbon

# ==============================================================================
# SEL INSTALASI - JALANKAN INI DULU
# ==============================================================================
# Uninstall versi yang mungkin konflik untuk memastikan instalasi bersih
!pip uninstall -y torch torchvision torchaudio neuralforecast pytorch-lightning lightning-utilities

# Install neuralforecast dengan dependensi modelnya.
# Pip akan secara otomatis memilih versi torch, torchvision, dll. yang kompatibel.
!pip install "neuralforecast[models]"

Found existing installation: torch 2.9.0
Uninstalling torch-2.9.0:
  Successfully uninstalled torch-2.9.0
Found existing installation: neuralforecast 3.1.2
Uninstalling neuralforecast-3.1.2:
  Successfully uninstalled neuralforecast-3.1.2
Found existing installation: pytorch-lightning 2.5.5
Uninstalling pytorch-lightning-2.5.5:
  Successfully uninstalled pytorch-lightning-2.5.5
Found existing installation: lightning-utilities 0.15.2
Uninstalling lightning-utilities-0.15.2:
  Successfully uninstalled lightning-utilities-0.15.2
  Using cached neuralforecast-3.1.2-py3-none-any.whl.metadata (14 kB)
  Using cached torch-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached pytorch_lightning-2.5.5-py3-none-any.whl.metadata (20 kB)
  Using cached lightning_utilities-0.15.2-py3-none-any.whl.metadata (5.7 kB)
Using cached pytorch_lightning-2.5.5-py3-none-any.whl (832 kB)
Using cached torch-2.9.0-cp312-cp312-manylinux_2_28_x86_64.whl (899.7 MB)
Using cached neuralforecast-3

In [ ]:
# ==============================================================================
# 1. Import Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import skew, kurtosis
from neuralforecast import NeuralForecast
# 1.1 Import Semua Model yang Akan Digunakan
from neuralforecast.models import NBEATS, NHITS, LSTM, TCN, NLinear, PatchTST, TiDE
# 1.2 Import Semua Loss Function yang Akan Digunakan
from neuralforecast.losses.pytorch import MAE, MSE, HuberLoss, MAPE, SMAPE, QuantileLoss
from sklearn.preprocessing import MinMaxScaler
import time
import psutil
import os
from codecarbon import EmissionsTracker
import random
import torch
import logging
import requests
from io import StringIO
import traceback
try:
    from google.colab import drive
    GOOGLE_COLAB = True
except ImportError:
    GOOGLE_COLAB = False

# ==============================================================================
# 2. Konfigurasi & Hyperparameters
# ==============================================================================
# 2.1 Pengaturan Reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# 2.2 Pengaturan Eksperimen Utama
SELECTED_DATASET_INDEX = 4
SELECTED_MODEL_INDEX = 1

# 2.3 Konfigurasi Detail Dataset
DATASET_CONFIG = {
    1: {'name': 'Bike Sharing', 'type': 'csv', 'source': "https://raw.githubusercontent.com/kanadakurniawan/loss-function-comparison/8ae1f330d2d94645a6b647ab357fa786a5e1f956/dataset/bike-sharing.csv", 'freq': 'H', 'value_column': 'cnt', 'time_column': 'datetime', 'series_selector': 0},
    2: {'name': 'Pasut BMKG', 'type': 'csv', 'source': 'https://raw.githubusercontent.com/kanadakurniawan/loss-function-comparison/6912fb758bcf4f6984c3d09c70dee9972b987a4b/dataset/db-2022-2024-pasut.csv', 'freq': 'H', 'value_column': 'pasut', 'time_column': 'datetime', 'series_selector': 0},
    3: {'name': 'Parking Birmingham', 'type': 'csv', 'source': 'https://raw.githubusercontent.com/kanadakurniawan/loss-function-comparison/8ae1f330d2d94645a6b647ab357fa786a5e1f956/dataset/parking-birmingham.csv', 'freq': '30min', 'value_column': 'Occupancy', 'time_column': 'datetime', 'series_selector': 0},
    4: {'name': 'Solar Power Generation', 'type': 'csv', 'source': 'https://raw.githubusercontent.com/kanadakurniawan/loss-function-comparison/8ae1f330d2d94645a6b647ab357fa786a5e1f956/dataset/Actual_31.85_-110.85_2006_UPV_100MW_5_Min.csv', 'freq': '5min', 'value_column': 'Power(MW)', 'time_column': 'datetime', 'series_selector': 0},
    5: {'name': 'Cacar Air Hungaria', 'type': 'csv', 'source': 'https://raw.githubusercontent.com/kanadakurniawan/loss-function-comparison/8ae1f330d2d94645a6b647ab357fa786a5e1f956/dataset/cacar-air-hungaria.csv', 'freq': 'W', 'value_column': 'BUDAPEST', 'time_column': 'datetime', 'series_selector': 0},
    6: {'name': 'M4 Hourly Dataset', 'type': 'tsf', 'source': 'https://raw.githubusercontent.com/kanadakurniawan/loss-function-comparison/8f605f9fbc107e6303174d3f615a5d591785d55e/dataset/m4_hourly_dataset.tsf', 'freq': 'H', 'parser_variant': 'standard', 'value_column': None, 'time_column': None, 'series_selector': 0}
}

# 2.4 Konfigurasi Detail Model
MODEL_CONFIG = {
    1: {'name': 'LSTM', 'class': LSTM}, 2: {'name': 'TCN', 'class': TCN},
    3: {'name': 'NBEATS', 'class': NBEATS}, 4: {'name': 'NHITS', 'class': NHITS},
    5: {'name': 'NLinear', 'class': NLinear}, 6: {'name': 'PatchTST', 'class': PatchTST},
    7: {'name': 'TiDE', 'class': TiDE}
}

# 2.5 Pengaturan Preprocessing
NAN_IMPUTATION_METHOD = 'ffill_bfill'

# 2.6 Pengaturan Model & Training
# >>> PERBAIKAN: Mengembalikan Horizon ke nilai yang bermakna untuk analisis stabilitas <<<
INPUT_WINDOW_SIZE = 336
HORIZON = 24
BATCH_SIZE = 32
VALIDATION_SIZE_FIT = HORIZON

# 2.6.1 Konfigurasi Training Epoch Tetap
FIXED_EPOCHS_PER_FOLD = 50
EARLY_STOP_PATIENCE_VALIDATIONS = 4

# 2.7 Pengaturan Cross-Validation
N_CROSSVALIDATION_FOLDS = 3
CV_STEP_SIZE = 24

# 2.8 Pengaturan Lainnya
os.environ['NIXTLA_ID_AS_COL'] = '1'
logging.getLogger("codecarbon").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)
logging.getLogger("lightning_fabric").setLevel(logging.WARNING)
GDRIVE_OUTPUT_FOLDER = "/content/drive/My Drive/S2/Thesis/loss-function-comparison/hasil/"

# ==============================================================================
# 3. Inisialisasi Lingkungan (Google Drive)
# ==============================================================================
if GOOGLE_COLAB:
    print("Mencoba mount Google Drive...")
    try:
        drive.mount('/content/drive', force_remount=True)
        print(f"Google Drive di-mount. Memeriksa/membuat folder output: {GDRIVE_OUTPUT_FOLDER}")
        os.makedirs(GDRIVE_OUTPUT_FOLDER, exist_ok=True)
        print(f"Folder output '{GDRIVE_OUTPUT_FOLDER}' siap.")
    except Exception as e:
        print(f"Gagal mount Google Drive atau membuat folder: {e}")
        print("Hasil TIDAK akan disimpan ke Google Drive.")
        GDRIVE_OUTPUT_FOLDER = None
else:
    print("Bukan lingkungan Google Colab, hasil tidak akan disimpan ke Google Drive.")
    GDRIVE_OUTPUT_FOLDER = None

# ==============================================================================
# 4. Fungsi Helper
# ==============================================================================
# (Semua fungsi helper dari jawaban sebelumnya ada di sini)
def load_data_from_source(source_path_or_url):
    """Membaca konten data dari URL atau file lokal."""
    source_str = str(source_path_or_url)
    if source_str.startswith('http'):
        print(f"Mengunduh data dari: {source_str}")
        try:
            response = requests.get(source_str, timeout=60)
            response.raise_for_status(); print("Data berhasil diunduh.")
            content_bytes = response.content; decoded_content = None; header_encoding = response.encoding
            encodings_to_try = ['utf-8', 'iso-8859-1', 'latin1', 'cp1252']
            if header_encoding and header_encoding.lower() not in [e.lower() for e in encodings_to_try]: encodings_to_try.insert(0, header_encoding)
            elif header_encoding:
                 try: idx = [e.lower() for e in encodings_to_try].index(header_encoding.lower()); encodings_to_try.insert(0, encodings_to_try.pop(idx))
                 except ValueError: encodings_to_try.insert(0, header_encoding)
            for enc in encodings_to_try:
                try: decoded_content = content_bytes.decode(enc); print(f"Konten decode dgn '{enc}'."); break
                except UnicodeDecodeError: continue
                except Exception as decode_err: print(f" Error decode dgn '{enc}': {decode_err}"); continue
            if decoded_content is None:
                print(f"Error: Gagal decode konten URL.")
                try: decoded_content = content_bytes.decode('utf-8', errors='replace'); print("Warn: Decode final dgn utf-8 replace.")
                except Exception: print("Gagal total decoding."); return None
            return decoded_content
        except requests.exceptions.RequestException as e: print(f"Error unduh data: {e}"); return None
        except Exception as e: print(f"Error lain proses URL: {e}"); print(traceback.format_exc()); return None
    else:
        print(f"Membaca data dari file lokal: {source_str}")
        try:
            if not os.path.exists(source_str):
                print(f"Error: File tdk ditemukan: {source_str}")
                if 'drive/My Drive' in source_str: print("Tips: Pastikan Drive ter-mount.")
                return None
            encodings_to_try = ['utf-8', 'iso-8859-1', 'latin1', 'cp1252']
            content = None
            for enc in encodings_to_try:
                 try:
                     with open(source_str, 'r', encoding=enc) as f: content = f.read()
                     print(f"Data dibaca (lokal) dgn encoding '{enc}'.")
                     break
                 except UnicodeDecodeError: pass
                 except Exception as e: print(f"Error baca file {source_str} dgn enc '{enc}': {e}"); return None
            if content is None: print("Error: Gagal baca file lokal dgn encoding yg dicoba."); return None
            return content
        except Exception as e: print(f"Error tak terduga baca file lokal {source_str}: {e}"); return None

def parse_tsf_data(raw_content, parser_variant):
    """Mem-parsing data TSF mentah (dari string)."""
    parsed_series, series_ids, start_times = [], [], []
    print(f"Memulai parsing TSF (varian: {parser_variant})...")
    lines = raw_content.splitlines(); reading_data = False; skipped_lines = 0; parsed_count = 0
    for i, line in enumerate(lines):
        line = line.strip()
        if not line or line.startswith(("#", "@relation", "@attribute", "@frequency", "@horizon", "@missing", "@equallength")): continue
        if line.startswith("@data"): reading_data = True; continue
        if reading_data:
            parts = line.split(":")
            try:
                if parser_variant == 'australia' and len(parts) >= 4:
                    series_name, state_name, start_time_str, values_str = parts[0], parts[1], parts[2], parts[3]; unique_id = f"{series_name}_{state_name}"
                elif parser_variant == 'standard' and len(parts) >= 3:
                    series_name, start_time_str, values_str = parts[0], parts[1], parts[2]; unique_id = series_name
                else: skipped_lines += 1; continue
                try: start_time = pd.Timestamp(start_time_str.replace(' ', 'T'))
                except ValueError:
                     try: start_time = pd.to_datetime(start_time_str, format='%Y-%m-%d %H-%M-%S')
                     except ValueError: skipped_lines += 1; continue
                time_series = []
                for val_str in values_str.split(","):
                    val_str = val_str.strip()
                    if val_str and val_str != '?':
                        try: time_series.append(float(val_str))
                        except ValueError: time_series.append(np.nan)
                    elif val_str == '?': time_series.append(np.nan)
                if time_series: parsed_series.append(time_series); series_ids.append(unique_id); start_times.append(start_time); parsed_count += 1
            except Exception as e: skipped_lines += 1; pass
    if skipped_lines > 0: print(f"Peringatan: Melewati {skipped_lines} baris saat parsing TSF.")
    print(f"Parsing TSF selesai. {parsed_count} series berhasil diparsing.")
    return series_ids, start_times, parsed_series

def parse_csv_data(raw_content, time_col, value_col, dataset_name):
    """Mem-parsing data CSV mentah (dari string)."""
    print(f"Memulai parsing CSV (time: '{time_col}', value: '{value_col}')...")
    try:
        df = pd.read_csv(StringIO(raw_content))
        if time_col not in df.columns: raise ValueError(f"Kolom waktu '{time_col}' tidak ada. Kolom: {df.columns.tolist()}")
        if value_col not in df.columns: raise ValueError(f"Kolom nilai '{value_col}' tidak ada. Kolom: {df.columns.tolist()}")
        try: df[time_col] = pd.to_datetime(df[time_col])
        except (ValueError, TypeError):
            print(f"Gagal parse '{time_col}' dgn format default, coba 'dd-mm-yyyy HH:MM'...")
            try: df[time_col] = pd.to_datetime(df[time_col], format='%d-%m-%Y %H:%M')
            except Exception as e_fmt: raise ValueError(f"Gagal parse '{time_col}' dgn format yg dicoba: {e_fmt}")
        df[value_col] = pd.to_numeric(df[value_col], errors='coerce')
        df = df.sort_values(by=time_col).reset_index(drop=True)
        start_time = df[time_col].iloc[0]; time_series = df[value_col].tolist()
        unique_id = f"{dataset_name.replace(' ', '_')}_Series"
        print(f"Parsing CSV selesai. 1 series, {len(time_series)} titik data.")
        return [unique_id], [start_time], [time_series]
    except Exception as e: print(f"Error parsing CSV: {e}"); print(traceback.format_exc()); return [], [], []

def load_and_parse_data(dataset_index, config):
    """Fungsi utama untuk memuat dan mem-parsing dataset berdasarkan Indeks."""
    cfg = config.get(dataset_index);
    if not cfg: raise ValueError(f"Indeks Dataset '{dataset_index}' tidak ditemukan.")
    dataset_name = cfg['name']
    print(f"\n-- Memuat Dataset {dataset_index}: {dataset_name} --")
    raw_content = load_data_from_source(cfg['source'])
    if raw_content is None: print(f"Gagal memuat konten untuk {dataset_name}."); return None, None, None, None, None
    data_freq = cfg['freq']
    if cfg['type'] == 'tsf': ids, starts, series_list = parse_tsf_data(raw_content, cfg['parser_variant'])
    elif cfg['type'] == 'csv': ids, starts, series_list = parse_csv_data(raw_content, cfg['time_column'], cfg['value_column'], dataset_name)
    else: raise ValueError(f"Tipe dataset '{cfg['type']}' tidak dikenal.")
    if not ids: print(f"Tidak ada series diparsing untuk {dataset_name}."); return None, None, None, None, None
    return ids, starts, series_list, data_freq, dataset_name

def select_series(all_ids, all_start_times, all_series_data, index_or_name, dataset_name):
    """Memilih time series spesifik dari hasil parsing berdasarkan indeks atau nama."""
    if not all_ids: raise ValueError("Tdk ada data series tersedia u/ dipilih.")
    selected_index = -1
    if isinstance(index_or_name, int):
        if 0 <= index_or_name < len(all_ids): selected_index = index_or_name
        else: raise ValueError(f"Indeks {index_or_name} di luar jangkauan (0-{len(all_ids) - 1}).")
    elif isinstance(index_or_name, str):
        try: selected_index = all_ids.index(index_or_name)
        except ValueError: raise ValueError(f"Nama series '{index_or_name}' tdk ditemukan di list ID: {all_ids}")
    else: raise TypeError("'index_or_name' hrs int/str.")

    selected_id = all_ids[selected_index]
    start_time = all_start_times[selected_index]
    time_series = all_series_data[selected_index]
    print(f"\nMemilih series: '{selected_id}' (Index: {selected_index}) dari '{dataset_name}'")
    if not time_series: print(f"Peringatan: Time series '{selected_id}' kosong."); return selected_id, start_time, []
    print(f"  -> {len(time_series)} titik data, mulai dari {start_time}.")
    return selected_id, start_time, time_series

def handle_nan_values(ts, method='ffill_bfill'):
    """Menangani nilai NaN dalam time series (input berupa list)."""
    if not isinstance(ts, (list, np.ndarray, pd.Series)): raise TypeError("Input 'ts' hrs list/array/Series.")
    if len(ts) == 0: print("Warn: TS kosong sblm handle NaN."); return []
    ts_series = pd.Series(ts, dtype=float); initial_nan_count = ts_series.isna().sum()
    if initial_nan_count == 0: print("Tidak ada nilai NaN."); return ts_series.tolist()
    print(f"Menangani {initial_nan_count}/{len(ts_series)} NaN dgn metode: {method}")
    if method == 'ffill_bfill': filled_ts = ts_series.ffill().bfill()
    elif method == 'mean': mean_val = ts_series.mean(); filled_ts = ts_series.fillna(mean_val if pd.notna(mean_val) else 0); print(f"  Imputasi mean: {mean_val:.4f}" if pd.notna(mean_val) else " (rata2 NaN)")
    elif method == 'median': median_val = ts_series.median(); filled_ts = ts_series.fillna(median_val if pd.notna(median_val) else 0); print(f"  Imputasi median: {median_val:.4f}" if pd.notna(median_val) else " (median NaN)")
    elif method == 'interpolate_linear': filled_ts = ts_series.interpolate(method='linear', limit_direction='both').ffill().bfill()
    else: print(f"Warn: Metode '{method}' tdk dikenal. Pakai 'ffill_bfill'."); filled_ts = ts_series.ffill().bfill()
    final_nan_count = filled_ts.isna().sum()
    if final_nan_count > 0: print(f"Warn: Masih ada {final_nan_count} NaN! Isi dgn 0."); filled_ts = filled_ts.fillna(0)
    else: print("Semua NaN berhasil ditangani.")
    return filled_ts.tolist()

def prepare_dataframe_for_neuralforecast(time_series, unique_id, start_time, freq):
    """Mempersiapkan Pandas DataFrame dalam format yang dibutuhkan NeuralForecast."""
    if not isinstance(time_series, (list, np.ndarray)): raise TypeError("time_series hrs list/array")
    if len(time_series) == 0: raise ValueError("time_series kosong")
    if not isinstance(start_time, pd.Timestamp):
         try: start_time = pd.Timestamp(start_time)
         except Exception as e: raise ValueError(f"start_time tdk valid: {start_time} - {e}")
    if not freq: raise ValueError("freq tdk boleh kosong/None")
    try: timestamps = pd.date_range(start=start_time, periods=len(time_series), freq=freq)
    except ValueError as e: raise ValueError(f"Gagal buat date_range: start={start_time}, periods={len(time_series)}, freq='{freq}': {e}")
    df = pd.DataFrame({"ds": timestamps, "y": time_series}); df["unique_id"] = unique_id
    df['y'] = df['y'].astype(float); return df

def create_timeseries_cv_folds(data, horizon, step_size, n_crossvalidation, freq, input_window_size):
    """Membagi data time series menjadi beberapa fold untuk cross-validation (sliding window)."""
    if not isinstance(data, np.ndarray):
        try: data = np.array(data, dtype=float)
        except ValueError: raise TypeError("Input 'data' CV hrs array float.")
    if np.isnan(data).any(): print("Warn: NaN di data input create_timeseries_cv_folds.")
    dataset_length = len(data);
    if dataset_length == 0: raise ValueError("Data input CV kosong.")
    if not all(isinstance(x, int) and x > 0 for x in [horizon, step_size, n_crossvalidation, input_window_size]): raise ValueError("Parameter CV hrs int positif.")
    total_test_step_length = horizon * n_crossvalidation + step_size * (n_crossvalidation - 1)
    min_length_needed = total_test_step_length + input_window_size
    if dataset_length < min_length_needed: raise ValueError(f"Dataset pendek ({dataset_length}) u/ CV. Butuh min {min_length_needed} (H={horizon},F={n_crossvalidation},S={step_size},I={input_window_size}).")
    train_window_length = dataset_length - total_test_step_length
    if train_window_length < input_window_size: raise ValueError(f"Train window ({train_window_length}) < input size ({input_window_size}).")
    print(f"\nMembuat {n_crossvalidation} fold CV:")
    print(f"  Freq={freq}, DataLen={dataset_length}, InputWin={input_window_size}, Horizon={horizon}, Step={step_size}, TrainWin={train_window_length}")
    folds = []
    for i in range(n_crossvalidation):
        start_train = i * step_size; end_train = start_train + train_window_length
        start_test = end_train; end_test = start_test + horizon
        if end_test > dataset_length: print(f"Warn: Fold {i+1} melebihi data. Stop."); break
        train_fold_data = data[start_train:end_train]; test_fold_data = data[start_test:end_test]
        if len(train_fold_data) == 0 or len(test_fold_data) == 0: print(f"Warn: Fold {i+1} kosong. Stop."); break
        folds.append((train_fold_data, test_fold_data))
        print(f"  Fold {i+1}: Train[{start_train}:{end_train}](len={len(train_fold_data)}), Test[{start_test}:{end_test}](len={len(test_fold_data)})")
    actual_folds_created = len(folds)
    if actual_folds_created == 0: raise RuntimeError("Gagal buat fold CV.")
    elif actual_folds_created < n_crossvalidation: print(f"\nPeringatan: Hanya {actual_folds_created}/{n_crossvalidation} fold dibuat.")
    return folds

def denormalize(data_normalized, scaler):
    """Mengembalikan data yang dinormalisasi ke skala aslinya."""
    if isinstance(data_normalized, (int, float)): data_normalized = np.array([data_normalized])
    elif isinstance(data_normalized, pd.Series): data_normalized = data_normalized.to_numpy()
    elif isinstance(data_normalized, list): data_normalized = np.array(data_normalized)
    if not isinstance(data_normalized, np.ndarray): raise TypeError("Input denorm hrs array.")
    is_scaler_valid = hasattr(scaler, 'scale_') and scaler.scale_ is not None and not np.all(scaler.scale_ == 0)
    if not is_scaler_valid:
        if hasattr(scaler, 'min_') and scaler.min_ is not None: return np.full(data_normalized.shape, scaler.min_[0])
        else: return data_normalized.flatten()
    try:
        if data_normalized.ndim == 1: data_reshaped = data_normalized.reshape(-1, 1)
        elif data_normalized.ndim == 2 and data_normalized.shape[1] == 1: data_reshaped = data_normalized
        else: raise ValueError(f"Input denorm hrs 1D/2D(1 col). Shape: {data_normalized.shape}")
        if np.isnan(data_reshaped).any(): print("Warn: NaN sblm inverse_transform.")
        data_denormalized = scaler.inverse_transform(data_reshaped)
        if np.isnan(data_denormalized).any(): print("Warn: NaN stlh inverse_transform.")
        return data_denormalized.flatten()
    except Exception as e: print(f"Error denorm: {e}. Input shape: {data_normalized.shape}"); print(traceback.format_exc()); return data_normalized.flatten()

def calculate_mase_scaling_factor(train_series):
    """Menghitung faktor skala MASE (MAE dari in-sample one-step naive forecast)."""
    if len(train_series) < 2: return 1.0
    train_series_pd = pd.Series(train_series)
    factor = np.mean(np.abs(train_series_pd.diff(1).dropna()))
    if pd.isna(factor) or factor < 1e-9: return 1e-9
    return factor

# ==============================================================================
# 5. Proses Utama
# ==============================================================================

# 5.1 Memuat dan Memparsing Data
try:
    all_ids, all_start_times, all_series_data, data_freq, dataset_name = load_and_parse_data(
        SELECTED_DATASET_INDEX, DATASET_CONFIG
    )
    if all_ids is None: exit(f"Gagal memuat/parsing Dataset {SELECTED_DATASET_INDEX}.")
except Exception as e: print(f"Error kritis load/parse: {e}"); print(traceback.format_exc()); exit()

# 5.2 Memilih, Membersihkan, dan Menganalisis Data Awal
try:
    selector = DATASET_CONFIG[SELECTED_DATASET_INDEX].get('series_selector', 0)
    selected_id, dataset_start_time, ts_raw = select_series(
        all_ids, all_start_times, all_series_data, selector, dataset_name
    )
    if ts_raw is None or not isinstance(ts_raw, list): raise ValueError("Data mentah (ts_raw) tidak valid.")
    if len(ts_raw) == 0: raise ValueError("Data mentah (ts_raw) kosong.")
    ts_cleaned = handle_nan_values(ts_raw, method=NAN_IMPUTATION_METHOD)
    if not ts_cleaned: raise ValueError("Data kosong setelah cleaning NaN.")
except (ValueError, IndexError, TypeError) as e: print(f"Error pilih/bersihkan seri: {e}"); exit()
except Exception as e: print(f"Error tak terduga pilih/bersihkan seri: {e}"); print(traceback.format_exc()); exit()

# 5.3 Normalisasi Data
print("\nNormalisasi data (MinMaxScaler [0, 1])...")
scaler = MinMaxScaler(feature_range=(0, 1))
try:
    ts_cleaned_array = np.array(ts_cleaned).reshape(-1, 1)
    if np.all(ts_cleaned_array == ts_cleaned_array[0]): print("Warn: Data konstan setelah cleaning.")
    ts_normalized = scaler.fit_transform(ts_cleaned_array).flatten()
    if np.isnan(ts_normalized).any(): print("Warn: NaN setelah normalisasi. Ganti dgn 0."); ts_normalized = np.nan_to_num(ts_normalized, nan=0.0)

    mase_scaling_factor_norm = calculate_mase_scaling_factor(ts_normalized)
    print(f"Faktor Skala MASE (dari data ternormalisasi): {mase_scaling_factor_norm:.6f}")

except Exception as e: print(f"Error normalisasi: {e}"); print(traceback.format_exc()); exit()

# 5.4 Perhitungan Training Steps & Pembuatan Fold CV
try:
    print("\nMembuat fold CV dgn parameter tetap...")
    folds = create_timeseries_cv_folds(
        ts_normalized, HORIZON, CV_STEP_SIZE, N_CROSSVALIDATION_FOLDS, data_freq, INPUT_WINDOW_SIZE
    )
    num_actual_folds = len(folds)

    # >>> PERBAIKAN: Logika Training Epoch Tetap <<<
    train_window_length = len(folds[0][0])
    batches_per_epoch = int(np.ceil(train_window_length / BATCH_SIZE))

    final_max_steps = FIXED_EPOCHS_PER_FOLD * batches_per_epoch

    # Validasi ~4x per epoch, tapi minimal 1x per epoch
    final_val_freq = max(1, int(batches_per_epoch / 4))
    # Patience dalam langkah, bukan jumlah validasi
    final_early_stop_patience_steps = EARLY_STOP_PATIENCE_VALIDATIONS * final_val_freq

    print("\n--- Konfigurasi Training Epoch Tetap Dihitung ---")
    print(f"  Panjang Train per Fold: {train_window_length}")
    print(f"  Batch per Epoch       : {batches_per_epoch}")
    print(f"  TARGET EPOCHS         : {FIXED_EPOCHS_PER_FOLD}")
    print(f"  FINAL Max Steps/Fold  : {final_max_steps}")
    print(f"  FINAL Freq. Validasi  : Setiap {final_val_freq} langkah")
    print(f"  FINAL Patience (steps): {final_early_stop_patience_steps} langkah")

except Exception as e: print(f"Error buat fold CV atau hitung steps: {e}"); print(traceback.format_exc()); exit()

# 5.5 Definisi Fungsi Loss dan Loop Utama Eksperimen
loss_functions_to_test = {
    "MSE": MSE(), "MAE": MAE(), "Huber": HuberLoss(delta=1.0),
    "MAPE": MAPE(), "SMAPE": SMAPE(), "Quantile0.1": QuantileLoss(0.1),
    "Quantile0.5": QuantileLoss(0.5), "Quantile0.9": QuantileLoss(0.9)
}

all_detailed_results_dfs = []
overall_experiment_start_time = time.time()
model_name = MODEL_CONFIG[SELECTED_MODEL_INDEX]['name']
model_class = MODEL_CONFIG[SELECTED_MODEL_INDEX]['class']

print(f"\n===== Memulai Eksperimen untuk Model: {model_name} =====")
for loss_name, current_loss_function in loss_functions_to_test.items():
    print(f"\n===== Menjalankan Eksperimen untuk Loss: {loss_name} =====")

    # 5.5.1 Inisialisasi Model per Loss
    print(f"\nInisialisasi model {model_name} dgn loss {loss_name}...")
    common_params = {
        'h': HORIZON, 'input_size': INPUT_WINDOW_SIZE, 'loss': current_loss_function,
        'max_steps': final_max_steps, 'batch_size': BATCH_SIZE, 'valid_loss': current_loss_function,
        'val_check_steps': final_val_freq,
        'early_stop_patience_steps': final_early_stop_patience_steps,
        'scaler_type': None, 'random_seed': seed
    }
    model_specific_params = {}
    model_params = {**common_params, **model_specific_params}
    try:
        model_instance = model_class(**model_params)
        nf = NeuralForecast(models=[model_instance], freq=data_freq)
        print(f"Model {model_name} & NeuralForecast diinisialisasi untuk loss {loss_name}.")
    except Exception as e:
        print(f"Error init model/NF untuk loss {loss_name}: {e}"); print(traceback.format_exc())
        print(f"!!! Melewati eksperimen untuk loss {loss_name} !!!")
        continue

    # 5.5.2 Loop Cross-Validation
    print(f"\nMemulai Cross-Validation untuk loss {loss_name}...")
    fold_start_time_cv = time.time()
    current_loss_metrics_data = []
    if loss_name == list(loss_functions_to_test.keys())[-1]:
        all_predictions_denorm = []
        all_actuals_denorm = []
    successful_folds_current_loss = 0

    for i, (train_fold_norm, test_fold_norm) in enumerate(folds):
        fold_num = i + 1
        print(f"\n--- Processing Fold {fold_num}/{num_actual_folds} (Loss: {loss_name}) ---")
        fold_start_time_fold = time.time()
        if len(train_fold_norm) < INPUT_WINDOW_SIZE: print(f"Error: Train fold {fold_num} < input size. Skip."); continue
        if len(test_fold_norm) == 0: print(f"Error: Test fold {fold_num} kosong. Skip."); continue
        if np.isnan(train_fold_norm).any() or np.isnan(test_fold_norm).any(): print(f"Warn: NaN di data fold {fold_num}.")

        fold_tracker = EmissionsTracker(measure_power_secs=1, log_level='warning', project_name=f"{dataset_name}_{model_name}_{loss_name}_Fold_{fold_num}")
        try: fold_tracker.start()
        except Exception as e: print(f"Warn: Gagal start CodeCarbon: {e}"); fold_tracker = None

        # Persiapan Data Fold
        try:
            current_freq = nf.freq
            if not current_freq: raise ValueError("Freq tidak valid atau None.")
            time_offset = pd.tseries.frequencies.to_offset(current_freq)
            train_start_index_global = i * CV_STEP_SIZE
            train_start_time_fold_ts = dataset_start_time + train_start_index_global * time_offset
            train_timestamps = pd.date_range(start=train_start_time_fold_ts, periods=len(train_fold_norm), freq=current_freq)
            test_start_time_fold_ts = train_timestamps[-1] + time_offset
            test_timestamps = pd.date_range(start=test_start_time_fold_ts, periods=len(test_fold_norm), freq=current_freq)
            train_df = prepare_dataframe_for_neuralforecast(train_fold_norm, selected_id, train_timestamps[0], current_freq)
            test_df_true = prepare_dataframe_for_neuralforecast(test_fold_norm, selected_id, test_timestamps[0], current_freq)
        except Exception as e:
            print(f"Error prep data fold {fold_num}: {e}"); print(traceback.format_exc()); print("Skip fold.")
            if fold_tracker:
                try: fold_tracker.stop()
                except Exception as se: print(f"Warn: Gagal stop tracker (error data prep): {se}")
            continue

        # Training Model
        print(f"Training model fold {fold_num} (Loss: {loss_name}, Train: {len(train_df)}, Val: {VALIDATION_SIZE_FIT}, MaxSteps: {final_max_steps})...")
        try:
            fit_output = nf.fit(df=train_df, val_size=VALIDATION_SIZE_FIT)
        except Exception as e:
            print(f"Error training fold {fold_num} (Loss: {loss_name}): {e}"); print(traceback.format_exc()); print("Skip fold.")
            if fold_tracker:
                try: fold_tracker.stop()
                except Exception as se: print(f"Warn: Gagal stop tracker (error training): {se}")
            continue

        # Prediksi
        print(f"Prediksi horizon {HORIZON} (Loss: {loss_name})...")
        try:
            forecast_df = nf.predict()
            if forecast_df is None or forecast_df.empty: raise ValueError("Hasil prediksi kosong.")
            forecast_df = forecast_df.reset_index()
        except Exception as e:
            print(f"Error prediksi fold {fold_num} (Loss: {loss_name}): {e}"); print(traceback.format_exc()); print("Skip fold.")
            if fold_tracker:
                try: fold_tracker.stop()
                except Exception as se: print(f"Warn: Gagal stop tracker (error prediksi): {se}")
            continue

        # Evaluasi Fold
        evaluation_successful = False
        mae, mse, mase, mae_naive = (np.nan,) * 4
        mean_res, std_res, min_res, max_res, max_abs_res, skew_res, kurt_res = (np.nan,) * 7
        actual_steps_taken = np.nan

        print(f"Evaluasi fold {fold_num} (Loss: {loss_name})...")
        try:
            y_true_normalized = test_df_true['y'].to_numpy()
            pred_col_name = model_name
            if pred_col_name not in forecast_df.columns:
                possible_cols = [col for col in forecast_df.columns if col.startswith(model_name)]
                if possible_cols: pred_col_name = possible_cols[0]; print(f"Warn: Guna kolom pred '{pred_col_name}'.")
                else: raise KeyError(f"Kolom pred '{model_name}'/* tdk ada. Kolom: {forecast_df.columns}")
            y_pred_normalized = forecast_df[pred_col_name].to_numpy()

            len_true, len_pred = len(y_true_normalized), len(y_pred_normalized)
            if len_pred != len_true:
                min_len = min(len_pred, len_true); print(f"Warn: Panjang pred ({len_pred}) != aktual ({len_true}). Adjust ke {min_len}.")
                if min_len == 0: raise ValueError("Data evaluasi 0 stlh adjust panjang.")
                y_pred_normalized=y_pred_normalized[:min_len]; y_true_normalized=y_true_normalized[:min_len]

            if loss_name == list(loss_functions_to_test.keys())[-1]:
                 y_true_denorm = denormalize(y_true_normalized, scaler)
                 y_pred_denorm = denormalize(y_pred_normalized, scaler)
                 all_actuals_denorm.append(y_true_denorm); all_predictions_denorm.append(y_pred_denorm)

            mae = mean_absolute_error(y_true_normalized, y_pred_normalized)
            mse = mean_squared_error(y_true_normalized, y_pred_normalized)

            last_train_val_norm = train_fold_norm[-1]
            y_pred_naive_norm = np.full_like(y_true_normalized, last_train_val_norm)
            mae_naive = mean_absolute_error(y_true_normalized, y_pred_naive_norm)
            mase = mae / mase_scaling_factor_norm if mase_scaling_factor_norm > 0 else np.inf

            residuals_norm = y_true_normalized - y_pred_normalized
            mean_res=np.mean(residuals_norm); std_res=np.std(residuals_norm)
            min_res=np.min(residuals_norm); max_res=np.max(residuals_norm)
            max_abs_res=np.max(np.abs(residuals_norm))
            skew_res=skew(residuals_norm); kurt_res=kurtosis(residuals_norm, fisher=True)
            if np.isnan(skew_res): skew_res = 0.0;
            if np.isnan(kurt_res): kurt_res = 0.0;

            current_model_instance = nf.models[0]
            if hasattr(current_model_instance, 'trainer') and hasattr(current_model_instance.trainer, 'global_step'):
                actual_steps_taken = current_model_instance.trainer.global_step

            evaluation_successful = True

        except Exception as eval_err:
            print(f"Error evaluasi fold {fold_num} (Loss: {loss_name}): {eval_err}"); print(traceback.format_exc()); print("Skip sisa evaluasi.")
            pass

        fold_end_time = time.time()
        fold_duration = fold_end_time - fold_start_time_fold

        fold_emissions = 0.0
        if fold_tracker:
            try:
                fold_emissions_data = fold_tracker.stop()
                if isinstance(fold_emissions_data, (int, float)): fold_emissions = float(fold_emissions_data)
                elif isinstance(fold_emissions_data, dict) and 'emissions' in fold_emissions_data: fold_emissions = float(fold_emissions_data['emissions'])
                if not isinstance(fold_emissions, float) or not np.isfinite(fold_emissions): fold_emissions = 0.0
            except Exception as e: fold_emissions = 0.0
        else: fold_emissions = 0.0

        if evaluation_successful:
            current_fold_metrics = (
                mae, mse, mase, mae_naive,
                mean_res, std_res, min_res, max_res, max_abs_res,
                skew_res, kurt_res,
                final_max_steps, actual_steps_taken, fold_duration, fold_emissions
            )
            current_loss_metrics_data.append(current_fold_metrics)
            successful_folds_current_loss += 1

            print(f"  Hasil Fold {fold_num}: [Sukses dievaluasi]")
            print(f"    Performa (Norm): MAE={mae:.6f}, MAE Naive={mae_naive:.6f}, MASE={mase:.4f}, MSE={mse:.6f}")
            print(f"    Stabilitas (Norm): MeanRes={mean_res:+.4f}, StdRes={std_res:.4f}, Skew={skew_res:.3f}, Kurt={kurt_res:.3f}")
            print(f"    Efisiensi: Durasi={fold_duration:.2f}s, CO2 Est.={fold_emissions:.6f} kg, Steps={actual_steps_taken}/{final_max_steps}")
        else:
            print(f"  Hasil Fold {fold_num}: [Evaluasi Gagal]")
            current_fold_metrics = (np.nan,) * 15
            current_loss_metrics_data.append(current_fold_metrics)


    # 5.5.3 Proses dan Simpan Hasil per Loss
    fold_end_time_cv = time.time()
    cv_duration_loss = fold_end_time_cv - fold_start_time_cv
    print(f"\nCross-Validation untuk loss '{loss_name}' selesai dalam {cv_duration_loss:.2f} detik. ({successful_folds_current_loss}/{num_actual_folds} fold sukses)")

    metric_names = [
        "MAE", "MSE", "MASE", "MAE Naive",
        "Mean Res", "Std Res", "Min Res", "Max Res", "Max Abs Res",
        "Skew Res", "Kurt Res",
        "Target Steps", "Actual Steps", "Duration (s)", "CO2 (kg)"
    ]
    results_df_loss = pd.DataFrame(current_loss_metrics_data, columns=metric_names)
    results_df_loss.index = range(1, num_actual_folds + 1)
    results_df_loss.index.name = "Fold"

    if successful_folds_current_loss > 0:
        mean_row = results_df_loss.mean(); std_row = results_df_loss.std()
        results_df_loss.loc['Rata-rata'] = mean_row
        results_df_loss.loc['Std Dev'] = std_row

    results_df_loss['Loss Function'] = loss_name
    all_detailed_results_dfs.append(results_df_loss)


# ==============================================================================
# 6. Ringkasan Akhir dan Penyimpanan
# ==============================================================================
# 6.1 Ringkasan Konsol
overall_end_time_all = time.time()
overall_duration_all = overall_end_time_all - overall_experiment_start_time
print(f"\n=============================================================")
print(f" SEMUA EKSPERIMEN FUNGSI LOSS SELESAI")
print(f"=============================================================")
print(f" Dataset Diproses   : {dataset_name} (Indeks: {SELECTED_DATASET_INDEX})")
print(f" Seri Dipilih       : '{selected_id}'")
print(f" Model Dasar        : {model_name}")
print(f" Parameter Tetap    : Horizon={HORIZON}, InputWindow={INPUT_WINDOW_SIZE}")
print(f" Strategi Training  : Epoch Tetap (Target: {FIXED_EPOCHS_PER_FOLD})")
print(f" Total Waktu Proses : {overall_duration_all:.2f} detik ({overall_duration_all/60:.2f} menit)")

# 6.2 Konsolidasi dan Penyimpanan Hasil
saved_files = []
if GDRIVE_OUTPUT_FOLDER and all_detailed_results_dfs:
    dataset_name_safe = "".join(c if c.isalnum() else "_" for c in dataset_name)

    # --- File 1: Hasil Detail Lengkap ---
    try:
        full_results_df = pd.concat(all_detailed_results_dfs).reset_index().set_index(['Loss Function', 'Fold'])
        detailed_filename = f"Hasil_Lengkap_{dataset_name_safe}_{model_name}.csv"
        detailed_filepath = os.path.join(GDRIVE_OUTPUT_FOLDER, detailed_filename)
        print(f"\nMenyimpan hasil detail lengkap ke: {detailed_filepath}")
        full_results_df.round(6).to_csv(detailed_filepath)
        saved_files.append(detailed_filepath)
        print("Berhasil disimpan.")
    except Exception as e:
        print(f"!!! GAGAL menyimpan file hasil detail: {e}"); print(traceback.format_exc())

    # --- File 2: Ringkasan Perbandingan ---
    try:
        summary_list = []
        for df_result in all_detailed_results_dfs:
            if 'Rata-rata' in df_result.index:
                summary_series = df_result.loc['Rata-rata']
                summary_series.name = df_result['Loss Function'].iloc[0]
                summary_list.append(summary_series)

        if summary_list:
            comparison_df = pd.DataFrame(summary_list)
            comparison_df.index.name = "Loss Function"

            comparison_filename = f"Ringkasan_Perbandingan_{dataset_name_safe}_{model_name}.csv"
            comparison_filepath = os.path.join(GDRIVE_OUTPUT_FOLDER, comparison_filename)
            print(f"\nMenyimpan ringkasan perbandingan loss ke: {comparison_filepath}")
            comparison_df.round(6).to_csv(comparison_filepath)
            saved_files.append(comparison_filepath)
            print("Berhasil disimpan.")

            print("\n--- Ringkasan Perbandingan Rata-Rata Metrik Antar Loss Function ---")
            pd.set_option('display.max_columns', None); pd.set_option('display.width', 120)
            print(comparison_df.round(6).to_string())
        else:
            print("\nTidak ada data rata-rata untuk membuat ringkasan perbandingan.")

    except Exception as e:
        print(f"!!! GAGAL membuat atau menyimpan file ringkasan perbandingan: {e}"); print(traceback.format_exc())

if saved_files:
    print("\nFile Hasil yang Telah Disimpan:")
    for file_path in saved_files: print(f"- {file_path}")
else:
    print("\nTidak ada file hasil yang berhasil disimpan.")


# ==============================================================================
# 7. Plotting (Opsional)
# ==============================================================================
print("\nMencoba plot fold terakhir sukses (dari Loss Function terakhir)...")
last_loss_results = all_detailed_results_dfs[-1] if all_detailed_results_dfs else pd.DataFrame()
successful_folds_last_loss = 0
if not last_loss_results.empty and 'Rata-rata' in last_loss_results.index:
     valid_fold_indices = [idx for idx in last_loss_results.index if isinstance(idx, int)]
     successful_folds_last_loss = len(valid_fold_indices)

if all_actuals_denorm and all_predictions_denorm and successful_folds_last_loss > 0:
    try:
        last_fold_actual = all_actuals_denorm[-1]; last_fold_pred = all_predictions_denorm[-1]
        last_successful_original_index = -1
        if successful_folds_last_loss > 0:
            successful_indices_last_loss = [idx for idx in last_loss_results.index if isinstance(idx, int)]
            if successful_indices_last_loss:
                last_successful_fold_num = successful_indices_last_loss[-1]
                last_successful_original_index = last_successful_fold_num - 1
        if last_successful_original_index == -1: raise ValueError("Tdk dpt tentukan idx asli fold sukses terakhir.")
        last_train_start_index_global = last_successful_original_index * CV_STEP_SIZE
        last_train_len = len(folds[last_successful_original_index][0])
        last_test_start_index_global = last_train_start_index_global + last_train_len
        current_freq = data_freq
        if not current_freq: raise ValueError("Freq tdk valid u/ plot.")
        time_offset = pd.tseries.frequencies.to_offset(current_freq)
        last_test_start_time = dataset_start_time + last_test_start_index_global * time_offset
        last_test_timestamps = pd.date_range(start=last_test_start_time, periods=len(last_fold_actual), freq=current_freq)
        plt.figure(figsize=(15, 7))
        plt.plot(last_test_timestamps, last_fold_actual, label=f'Aktual (Fold Asli #{last_successful_original_index+1})', marker='.', linewidth=1)
        plt.plot(last_test_timestamps, last_fold_pred, label=f'Prediksi {model_name} (Loss Terakhir, Fold #{last_successful_original_index+1})', marker='.', linestyle='--', linewidth=1)
        plt.title(f'Prediksi vs Aktual - Fold Terakhir ({selected_id}) - Dataset: {dataset_name} - Loss: {list(loss_functions_to_test.keys())[-1]}')
        plt.xlabel('Timestamp'); plt.ylabel('Nilai (Skala Asli)'); plt.legend(); plt.grid(True); plt.xticks(rotation=30); plt.tight_layout(); plt.show()
    except Exception as e: print(f"\nError buat plot: {e}"); print(traceback.format_exc())
else: print("\nTidak ada data dari fold sukses (di loss terakhir) untuk diplot.")

print("\n================ SEMUA EKSPERIMEN SELESAI ================")

Mencoba mount Google Drive...
Mounted at /content/drive
Google Drive di-mount. Memeriksa/membuat folder output: /content/drive/My Drive/S2/Thesis/loss-function-comparison/hasil/
Folder output '/content/drive/My Drive/S2/Thesis/loss-function-comparison/hasil/' siap.

-- Memuat Dataset 4: Solar Power Generation --
Mengunduh data dari: https://raw.githubusercontent.com/kanadakurniawan/loss-function-comparison/8ae1f330d2d94645a6b647ab357fa786a5e1f956/dataset/Actual_31.85_-110.85_2006_UPV_100MW_5_Min.csv
Data berhasil diunduh.
Konten decode dgn 'utf-8'.
Memulai parsing CSV (time: 'datetime', value: 'Power(MW)')...


[codecarbon WARNING @ 17:17:20] Multiple instances of codecarbon are allowed to run at the same time.


Parsing CSV selesai. 1 series, 52128 titik data.

Memilih series: 'Solar_Power_Generation_Series' (Index: 0) dari 'Solar Power Generation'
  -> 52128 titik data, mulai dari 2006-01-01 00:00:00.
Tidak ada nilai NaN.

Normalisasi data (MinMaxScaler [0, 1])...
Faktor Skala MASE (dari data ternormalisasi): 0.009552

Membuat fold CV dgn parameter tetap...

Membuat 3 fold CV:
  Freq=5min, DataLen=52128, InputWin=336, Horizon=24, Step=24, TrainWin=52008
  Fold 1: Train[0:52008](len=52008), Test[52008:52032](len=24)
  Fold 2: Train[24:52032](len=52008), Test[52032:52056](len=24)
  Fold 3: Train[48:52056](len=52008), Test[52056:52080](len=24)

--- Konfigurasi Training Epoch Tetap Dihitung ---
  Panjang Train per Fold: 52008
  Batch per Epoch       : 1626
  TARGET EPOCHS         : 50
  FINAL Max Steps/Fold  : 81300
  FINAL Freq. Validasi  : Setiap 406 langkah
  FINAL Patience (steps): 1624 langkah

===== Memulai Eksperimen untuk Model: LSTM =====

===== Menjalankan Eksperimen untuk Loss: MSE ===

[codecarbon WARNING @ 17:17:21] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 17:17:21] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon WARNING @ 17:17:21] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 17:17:21] Unable to access geographical location through primary API. Will resort to using the backup API - Exception : HTTPSConnectionPool(host='get.geojs.io', port=443): Read timed out. (read timeout=0.5) - url=https://get.geojs.io/v1/ip/geo.json
[codecarbon WARNING @ 17:17:22] Unable to access geographical location. Using 'Canada' as the default value - Exception : 'country' - url=https://get.geojs.io/v1/ip/geo.json


Training model fold 1 (Loss: MSE, Train: 52008, Val: 24, MaxSteps: 81300)...


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]